In [178]:
# import libraries needed

import pandas as pd
from pandas import DataFrame
import numpy as np
import plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import plotly.express as px
plotly.offline.init_notebook_mode(connected=True)
import geoip2.database
from geopy.geocoders import Nominatim
import plotly.figure_factory as ff

In [179]:
init_notebook_mode(connected = True)

In [180]:
# load dataset which will be processed

cwur = pd.read_csv("../dataset/cwurData.csv")
expenditure = pd.read_csv("../dataset/education_expenditure_supplementary_data.csv",lineterminator='\r')
country = pd.read_csv("../dataset/school_and_country_table.csv")
times = pd.read_csv("../dataset/timesData.csv")
shanghai = pd.read_csv("../dataset/shanghaiData.csv")


### **country**

In [181]:
country = country.rename(columns={'school_name':'university_name'})

In [182]:
lon_country = []
lat_country = []
for i in range(818):
    address = country["university_name"][i]
    gps = Nominatim(user_agent='myuseragent')
    location = gps.geocode(address,timeout=10000)
    if  location is not None:
        lon_country.append(location.longitude)
        lat_country.append(location.latitude)
    else:
        address_country = country["country"][i]
        gps = Nominatim(user_agent='myuseragent')
        location_country = gps.geocode(address_country,timeout=10000)
        lon_country.append(location_country.longitude)
        lat_country.append(location_country.latitude)

In [183]:
country["lon"] = lon_country
country["lat"] = lat_country

In [246]:
token = "pk.eyJ1IjoibHluMTIyMzExIiwiYSI6ImNrZ25vZ2dlcTBpY2YydHAxbHE4enBreHUifQ.WGPAq210C4NPoGRJG1j4HA"

fig = px.scatter_mapbox(country, lat="lat", lon="lon", hover_name="university_name", hover_data=[ "country"],
                        color_discrete_sequence=["slateblue"], zoom=3)
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig.write_html("country.html")

### **cwur rank**

In [193]:
# check information of the dataset

cwur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            2200 non-null   int64  
 1   institution           2200 non-null   object 
 2   country               2200 non-null   object 
 3   national_rank         2200 non-null   int64  
 4   quality_of_education  2200 non-null   int64  
 5   alumni_employment     2200 non-null   int64  
 6   quality_of_faculty    2200 non-null   int64  
 7   publications          2200 non-null   int64  
 8   influence             2200 non-null   int64  
 9   citations             2200 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2200 non-null   int64  
 12  score                 2200 non-null   float64
 13  year                  2200 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 240.8+ KB


In [194]:
cwur = cwur.rename(columns={'institution':'university_name'})

In [195]:
# check how much years in the dataset

cwur["year"].value_counts()

2015    1000
2014    1000
2013     100
2012     100
Name: year, dtype: int64

In [196]:
# split the dataframe by different years and obtain their first 50 schools

cwur_2015 = cwur[cwur["year"] == 2015]
cwur_2014 = cwur[cwur["year"] == 2014]
cwur_2013 = cwur[cwur["year"] == 2013]
cwur_2012 = cwur[cwur["year"] == 2012]

cwur_2015 = cwur_2015.sort_values(by=["world_rank"]).head(50)
cwur_2014 = cwur_2014.sort_values(by=["world_rank"]).head(50)
cwur_2013 = cwur_2013.sort_values(by=["world_rank"]).head(50)
cwur_2012 = cwur_2012.sort_values(by=["world_rank"]).head(50)

In [197]:
cwur_top50 = pd.concat([cwur_2012, cwur_2013,cwur_2014,cwur_2015], ignore_index=True)

In [198]:
cwur_top50.head(60)

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012
5,6,Princeton University,USA,5,8,14,2,53,33,26,NaN,101,82.50,2012
6,7,University of Oxford,United Kingdom,2,13,28,9,15,13,19,NaN,26,82.34,2012
7,8,Yale University,USA,6,14,31,12,14,6,15,NaN,66,79.14,2012
8,9,Columbia University,USA,7,23,21,10,13,12,14,NaN,5,78.86,2012
9,10,"University of California, Berkeley",USA,8,16,52,6,6,5,3,NaN,16,78.55,2012


In [199]:
lon_cwur = []
lat_cwur = []
for i in range(len(cwur_top50["university_name"])):
    address = cwur_top50["university_name"][i]
    gps = Nominatim(user_agent='myuseragent')
    location = gps.geocode(address,timeout=10000)
    if  location is not None:
        lon_cwur.append(location.longitude)
        lat_cwur.append(location.latitude)
    else:
        address_country = cwur_top50["country"][i]
        gps = Nominatim(user_agent='myuseragent')
        location_country = gps.geocode(address_country,timeout=10000)
        lon_cwur.append(location_country.longitude)
        lat_cwur.append(location_country.latitude)

In [200]:
cwur_top50["lon"]=lon_cwur
cwur_top50["lat"]=lat_cwur

In [201]:
cwur_top50.head(60)

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,lon,lat
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,-71.126782,42.367909
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,-71.095678,42.358396
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,-122.169365,37.431314
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,0.119739,52.199852
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,-118.125288,34.137014
5,6,Princeton University,USA,5,8,14,2,53,33,26,NaN,101,82.50,2012,-74.657635,40.346995
6,7,University of Oxford,United Kingdom,2,13,28,9,15,13,19,NaN,26,82.34,2012,-1.255668,51.758708
7,8,Yale University,USA,6,14,31,12,14,6,15,NaN,66,79.14,2012,-72.989670,41.257131
8,9,Columbia University,USA,7,23,21,10,13,12,14,NaN,5,78.86,2012,-73.961797,40.807949
9,10,"University of California, Berkeley",USA,8,16,52,6,6,5,3,NaN,16,78.55,2012,-122.266399,37.870946


In [202]:
cwur_top50[["patents"]] = cwur_top50[["patents"]].astype(int)

In [304]:
token = "pk.eyJ1IjoibHluMTIyMzExIiwiYSI6ImNrZ25vZ2dlcTBpY2YydHAxbHE4enBreHUifQ.WGPAq210C4NPoGRJG1j4HA"

fig = px.scatter_mapbox (cwur_top50, lat="lat", lon="lon", hover_name="university_name", hover_data=[ "world_rank","national_rank","score","quality_of_education","alumni_employment","quality_of_faculty","publications","influence","citations","patents"]
                        ,size="patents",animation_frame="year",color_discrete_sequence=["fuchsia"], size_max=50,zoom=2)
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig.write_html("cwur_top50.html",auto_play=False)

In [305]:
fig = plotly.express.bar(data_frame=cwur_top50, x="world_rank", y="university_name", 
                                      color="country",hover_name="country", hover_data=["score"],animation_frame="year")
fig.show()
fig.write_html("cwur_top50_rank.html",auto_play=False)

In [308]:
data2015 = cwur_2015.loc[:,["quality_of_education","citations", "score"]]
data2015["index"] = np.arange(1,len(data2015)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(data2015, diag='box', index='index',title="cwur_2015",colormap='Portland',
                                  colormap_type='cat',
                                  height=700, width=700)
iplot(fig)
fig.write_html("cwur_2015.html")

In [309]:
data2014 = cwur_2014.loc[:,["quality_of_education","citations", "score"]]
data2014["index"] = np.arange(1,len(data2014)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(data2014, diag='box', index='index',title="cwur_2014",colormap='Portland',
                                  colormap_type='cat',
                                  height=700, width=700)
iplot(fig)
fig.write_html("cwur_2014.html")

In [310]:
data2013 = cwur_2013.loc[:,["quality_of_education","citations", "score"]]
data2013["index"] = np.arange(1,len(data2013)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(data2013, diag='box', index='index',title="cwur_2013",colormap='Portland',
                                  colormap_type='cat',
                                  height=700, width=700)
iplot(fig)
fig.write_html("cwur_2013.html")

In [311]:
data2012 = cwur_2012.loc[:,["quality_of_education","citations", "score"]]
data2012["index"] = np.arange(1,len(data2012)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(data2012, diag='box', index='index',title="cwur_2012",colormap='Portland',
                                  colormap_type='cat',
                                  height=700, width=700)
iplot(fig)
fig.write_html("cwur_2012.html")

### **times rank**

In [251]:
# check information of the dataset

times.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2603 non-null   object 
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2603 non-null   object 
 8   total_score             2603 non-null   object 
 9   num_students            2544 non-null   object 
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   object 
 12  female_male_ratio       2370 non-null   object 
 13  year                    2603 non-null   int64  
dtypes: float64(4), int64(1), object(9)
memor

In [252]:
# check how much years in the dataset

times["year"].value_counts()

2016    800
2012    402
2015    401
2013    400
2014    400
2011    200
Name: year, dtype: int64

In [253]:
#  split the dataframe by different years

times_2016 = times[times["year"] == 2016]
times_2015 = times[times["year"] == 2015]
times_2014 = times[times["year"] == 2014]
times_2013 = times[times["year"] == 2013]
times_2012 = times[times["year"] == 2012]
times_2011 = times[times["year"] == 2011]

In [254]:
times_2016 = times_2016.head(50)
times_2015 = times_2015.head(50)
times_2014 = times_2014.head(50)
times_2013 = times_2013.head(50)
times_2012 = times_2012.head(50)
times_2011 = times_2011.head(50)

In [255]:
times_top50 = pd.concat([times_2011, times_2012,times_2013,times_2014,times_2015,times_2016], ignore_index=True)

In [256]:
lon_times = []
lat_times = []
for i in range(len(times_top50["university_name"])):
    address = times_top50["university_name"][i]
    gps = Nominatim(user_agent='myuseragent')
    location = gps.geocode(address,timeout=10000)
    if  location is not None:
        lon_times.append(location.longitude)
        lat_times.append(location.latitude)
    else:
        address_country = times_top50["country"][i]
        gps = Nominatim(user_agent='myuseragent')
        location_country = gps.geocode(address_country,timeout=10000)
        lon_times.append(location_country.longitude)
        lat_times.append(location_country.latitude)

In [257]:
times_top50["lon"]=lon_times
times_top50["lat"]=lat_times

In [258]:
times_top50.head(60)

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,lon,lat
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011,-71.126782,42.367909
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011,-118.125288,34.137014
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011,-71.095678,42.358396
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011,-122.169365,37.431314
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011,-74.657635,40.346995
5,6,University of Cambridge,United Kingdom,90.5,77.7,94.1,94.0,57.0,91.2,"18,812",11.8,34%,46 : 54,2011,0.119739,52.199852
6,6,University of Oxford,United Kingdom,88.2,77.2,93.9,95.1,73.5,91.2,"19,919",11.6,34%,46 : 54,2011,-1.255668,51.758708
7,8,"University of California, Berkeley",United States of America,84.2,39.6,99.3,97.8,-,91.1,"36,186",16.4,15%,50 : 50,2011,-122.266399,37.870946
8,9,Imperial College London,United Kingdom,89.2,90.0,94.5,88.3,92.9,90.6,"15,060",11.7,51%,37 : 63,2011,-0.174629,51.498983
9,10,Yale University,United States of America,92.1,59.2,89.7,91.5,-,89.5,"11,751",4.4,20%,50 : 50,2011,-72.989670,41.257131


In [259]:
times_top50[["total_score"]] = times_top50[["total_score"]].astype(float)

In [260]:
token = "pk.eyJ1IjoibHluMTIyMzExIiwiYSI6ImNrZ25vZ2dlcTBpY2YydHAxbHE4enBreHUifQ.WGPAq210C4NPoGRJG1j4HA"

fig = px.scatter_mapbox (times_top50, lat="lat", lon="lon", hover_name="university_name", hover_data=[ "world_rank","international","teaching","research","citations","income","num_students","student_staff_ratio","international_students","total_score"]
                        ,size="total_score",animation_frame="year",color_discrete_sequence=["royalblue"],zoom=2)
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig.write_html("times_top50.html",auto_play=False)

In [298]:
fig = plotly.express.bar(data_frame=times_top50, x="world_rank", y="university_name", 
                                      color="country",hover_name="university_name", hover_data=["total_score"],animation_frame="year")
fig.show()
fig.write_html("times_top50_rank.html",auto_play=False)

In [312]:
data2016 = times_2016.loc[:,["teaching","citations", "total_score"]]
data2016["index"] = np.arange(1,len(data2016)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(data2016, diag='box', index='index',title="times_2016",colormap='Portland',
                                  colormap_type='cat',
                                  height=700, width=700)
iplot(fig)
fig.write_html("times_2016.html")

In [313]:
data2015 = times_2015.loc[:,["teaching","citations", "total_score"]]
data2015["index"] = np.arange(1,len(data2015)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(data2015, diag='box', index='index',title="times_2015",colormap='Portland',
                                  colormap_type='cat',
                                  height=700, width=700)
iplot(fig)
fig.write_html("times_2015.html")

In [314]:
data2014 = times_2014.loc[:,["teaching","citations", "total_score"]]
data2014["index"] = np.arange(1,len(data2014)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(data2014, diag='box', index='index',title="times_2014",colormap='Portland',
                                  colormap_type='cat',
                                  height=700, width=700)
iplot(fig)
fig.write_html("times_2014.html")

In [315]:
data2013 = times_2013.loc[:,["teaching","citations", "total_score"]]
data2013["index"] = np.arange(1,len(data2013)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(data2013, diag='box', index='index',title="times_2013",colormap='Portland',
                                  colormap_type='cat',
                                  height=700, width=700)
iplot(fig)
fig.write_html("times_2013.html")

In [316]:
data2012 = times_2012.loc[:,["teaching","citations", "total_score"]]
data2012["index"] = np.arange(1,len(data2012)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(data2012, diag='box', index='index',title="times_2012",colormap='Portland',
                                  colormap_type='cat',
                                  height=700, width=700)
iplot(fig)
fig.write_html("times_2012.html")

In [317]:
data2011 = times_2011.loc[:,["teaching","citations", "total_score"]]
data2011["index"] = np.arange(1,len(data2011)+1)
# scatter matrix
fig = ff.create_scatterplotmatrix(data2011, diag='box', index='index',title="times_2011",colormap='Portland',
                                  colormap_type='cat',
                                  height=700, width=700)
iplot(fig)
fig.write_html("times_2011.html")

### **shanghai rank**

In [267]:
# check information of the dataset

shanghai.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4897 entries, 0 to 4896
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   world_rank       4897 non-null   object 
 1   university_name  4896 non-null   object 
 2   national_rank    4896 non-null   object 
 3   total_score      1101 non-null   float64
 4   alumni           4896 non-null   float64
 5   award            4895 non-null   float64
 6   hici             4895 non-null   float64
 7   ns               4875 non-null   float64
 8   pub              4895 non-null   float64
 9   pcp              4895 non-null   float64
 10  year             4897 non-null   int64  
dtypes: float64(7), int64(1), object(3)
memory usage: 421.0+ KB


In [268]:
# check how much years in the dataset

shanghai["year"].value_counts()

2007    510
2008    503
2009    501
2015    500
2011    500
2014    500
2010    500
2006    500
2005    500
2012    284
2013     99
Name: year, dtype: int64

In [269]:
#  split the dataframe by different years

shanghai_2015 = shanghai[shanghai["year"] == 2015]
shanghai_2014 = shanghai[shanghai["year"] == 2014]
shanghai_2013 = shanghai[shanghai["year"] == 2013]
shanghai_2012 = shanghai[shanghai["year"] == 2012]
shanghai_2011 = shanghai[shanghai["year"] == 2011]
shanghai_2010 = shanghai[shanghai["year"] == 2010]
shanghai_2009 = shanghai[shanghai["year"] == 2009]
shanghai_2008 = shanghai[shanghai["year"] == 2008]
shanghai_2007 = shanghai[shanghai["year"] == 2007]
shanghai_2006 = shanghai[shanghai["year"] == 2006]
shanghai_2005 = shanghai[shanghai["year"] == 2005]

shanghai_2015 = shanghai_2015.head(50)
shanghai_2014 = shanghai_2014.head(50)
shanghai_2013 = shanghai_2013.head(50)
shanghai_2012 = shanghai_2012.head(50)
shanghai_2011 = shanghai_2011.head(50)
shanghai_2010 = shanghai_2010.head(50)
shanghai_2009 = shanghai_2009.head(50)
shanghai_2008 = shanghai_2008.head(50)
shanghai_2007 = shanghai_2007.head(50)
shanghai_2006 = shanghai_2006.head(50)
shanghai_2005 = shanghai_2005.head(50)

In [270]:
shanghai_top50 = pd.concat([shanghai_2005, shanghai_2006,shanghai_2007,shanghai_2008,shanghai_2009,shanghai_2010,shanghai_2011,shanghai_2012,shanghai_2013,shanghai_2014,shanghai_2015], ignore_index=True)

In [271]:
shanghai_top50.head(60)

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005
5,6,California Institute of Technology,5,67.1,59.2,68.6,59.8,65.8,52.5,100.0,2005
6,7,Columbia University,6,62.3,79.4,60.6,56.1,54.2,69.5,45.4,2005
7,8,Princeton University,7,60.9,63.4,76.8,60.9,48.7,48.5,59.1,2005
8,9,University of Chicago,8,60.1,75.6,81.9,50.3,44.7,56.4,42.2,2005
9,10,University of Oxford,2,59.7,64.3,59.1,48.4,55.6,68.4,53.2,2005


In [272]:
lon_shanghai = []
lat_shanghai = []
list_no_location =[]

for i in range(len(shanghai_top50["university_name"])):
    address = shanghai_top50["university_name"][i]
    gps = Nominatim(user_agent='myuseragent')
    location = gps.geocode(address,timeout=10000)
    if  location is not None:
        lon_shanghai.append(location.longitude)
        lat_shanghai.append(location.latitude)
    else:
        list_no_location.append(shanghai_top50["university_name"][i])
        lon_shanghai.append("null")
        lat_shanghai.append("null")

In [273]:
shanghai_top50["lon"] = lon_shanghai
shanghai_top50["lat"]=lat_shanghai

In [274]:
no_location = list(set(list_no_location))

In [275]:
no_location 

['The Imperial College of Science, Technology and Medicine',
 'Rutgers, The State University of New Jersey',
 'Pierre and Marie  Curie University - Paris 6',
 'University of Paris Sud (Paris 11)',
 'Rutgers, The State University of New Jersey - New Brunswick',
 'University of Colorado at Boulder',
 'Karolinska Institute',
 'University of Minnesota, Twin Cities',
 'University of Paris-Sud (Paris 11)',
 'Swiss Federal Institute of Technology Zurich']

In [276]:
#shanghai_top50["university_name"]
shanghai_top50['lon'][shanghai_top50['university_name'] == 'Rutgers, The State University of New Jersey - New Brunswick'] = -73.99100129849981
shanghai_top50['lat'][shanghai_top50['university_name'] == 'Rutgers, The State University of New Jersey - New Brunswick'] = 40.71129035

shanghai_top50['lon'][shanghai_top50['university_name'] == 'Pierre and Marie  Curie University - Paris 6'] = 2.3760013207351394
shanghai_top50['lat'][shanghai_top50['university_name'] == 'Pierre and Marie  Curie University - Paris 6'] = 48.827222

shanghai_top50['lon'][shanghai_top50['university_name'] == 'University of Minnesota, Twin Cities'] = -93.23708812629405
shanghai_top50['lat'][shanghai_top50['university_name'] == 'University of Minnesota, Twin Cities'] = 44.97308605

shanghai_top50['lon'][shanghai_top50['university_name'] == 'Rutgers, The State University of New Jersey'] = -73.99100129849981
shanghai_top50['lat'][shanghai_top50['university_name'] == 'Rutgers, The State University of New Jersey'] = 40.71129035

shanghai_top50['lon'][shanghai_top50['university_name'] == 'University of Colorado at Boulder'] = -104.9376995
shanghai_top50['lat'][shanghai_top50['university_name'] == 'University of Colorado at Boulder'] = 39.6796152

shanghai_top50['lon'][shanghai_top50['university_name'] == 'University of Paris-Sud (Paris 11)'] = 2.3514616
shanghai_top50['lat'][shanghai_top50['university_name'] == 'University of Paris-Sud (Paris 11)'] = 48.8566969

shanghai_top50['lon'][shanghai_top50['university_name'] == 'Karolinska Institute'] = 18.609554
shanghai_top50['lat'][shanghai_top50['university_name'] == 'Karolinska Institute'] = 48.0308859

shanghai_top50['lon'][shanghai_top50['university_name'] == 'University of Paris Sud (Paris 11)'] = 2.3514616
shanghai_top50['lat'][shanghai_top50['university_name'] == 'University of Paris Sud (Paris 11)'] = 48.8566969

shanghai_top50['lon'][shanghai_top50['university_name'] == 'Swiss Federal Institute of Technology Zurich'] = 9.806297279860424
shanghai_top50['lat'][shanghai_top50['university_name'] == 'Swiss Federal Institute of Technology Zurich'] = 46.832989999999995

shanghai_top50['lon'][shanghai_top50['university_name'] == 'The Imperial College of Science, Technology and Medicine'] = -3.2765753
shanghai_top50['lat'][shanghai_top50['university_name'] == 'The Imperial College of Science, Technology and Medicine'] = 54.7023545



In [277]:
shanghai_top50.head(60)

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,lon,lat
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,-71.1268,42.3679
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,0.119739,52.1999
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,-122.169,37.4313
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,-122.266,37.8709
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005,-71.0957,42.3584
5,6,California Institute of Technology,5,67.1,59.2,68.6,59.8,65.8,52.5,100.0,2005,-118.125,34.137
6,7,Columbia University,6,62.3,79.4,60.6,56.1,54.2,69.5,45.4,2005,-73.9618,40.8079
7,8,Princeton University,7,60.9,63.4,76.8,60.9,48.7,48.5,59.1,2005,-74.6576,40.347
8,9,University of Chicago,8,60.1,75.6,81.9,50.3,44.7,56.4,42.2,2005,-87.5939,41.7854
9,10,University of Oxford,2,59.7,64.3,59.1,48.4,55.6,68.4,53.2,2005,-1.25567,51.7587


In [278]:
token = "pk.eyJ1IjoibHluMTIyMzExIiwiYSI6ImNrZ25vZ2dlcTBpY2YydHAxbHE4enBreHUifQ.WGPAq210C4NPoGRJG1j4HA"

fig = px.scatter_mapbox (shanghai_top50, lat="lat", lon="lon", hover_name="university_name", hover_data=[ "world_rank","national_rank","alumni","award","hici","ns","pub","pcp","total_score"]
                       ,size="award",animation_frame="year",color_discrete_sequence=["orange"],zoom=2)
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig.write_html("shanghai_top50.html",auto_play=False)

In [303]:
shanghai_top50_country = shanghai_top50.merge(country)

fig = plotly.express.bar(data_frame=shanghai_top50_country, x="world_rank", y="university_name", 
                                      color="country",hover_name="university_name", hover_data=["total_score"],animation_frame="year")
fig.show()
fig.write_html("shanghai_top50_rank.html",auto_play=False)

### **Expenditure and  World rank**

In [158]:
expenditure = pd.read_csv("../dataset/education_expenditure_supplementary_data.csv",lineterminator='\r')

In [159]:
expenditure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  333 non-null    object 
 1   institute_type           333 non-null    object 
 2   direct_expenditure_type  333 non-null    object 
 3   1995                     94 non-null     float64
 4   2000                     99 non-null     float64
 5   2005                     106 non-null    float64
 6   2009                     103 non-null    float64
 7   2010                     101 non-null    float64
 8   2011                     282 non-null    float64
dtypes: float64(6), object(3)
memory usage: 23.5+ KB


In [160]:
expenditure.head(10)

,country,institute_type,direct_expenditure_type,1995,2000,2005,2009,2010,2011
0,OECD Average,All Institutions,Public,4.9,4.9,5.0,5.4,5.4,5.3
1,Australia,All Institutions,Public,4.5,4.6,4.3,4.5,4.6,4.3
2,Austria,All Institutions,Public,5.3,5.4,5.2,5.7,5.6,5.5
3,Belgium,All Institutions,Public,5.0,5.1,5.8,6.4,6.4,6.4
4,Canada,All Institutions,Public,5.8,5.2,4.8,5.0,5.2,NaN
5,Chile,All Institutions,Public,NaN,4.2,3.3,4.1,4.3,3.9
6,Czech Republic,All Institutions,Public,4.8,4.2,4.1,4.2,4.1,4.4
7,Denmark,All Institutions,Public,6.5,6.4,6.8,7.5,7.6,7.5
8,Estonia,All Institutions,Public,NaN,NaN,4.7,5.9,5.6,5.2
9,Finland,All Institutions,Public,6.6,5.5,5.9,6.3,6.4,6.3


In [161]:
expenditure_total = expenditure[expenditure["direct_expenditure_type"] == "Total"]

In [162]:
expenditure_total = expenditure_total.fillna(0.000)

In [163]:
expenditure_mean = []
for i  in range(len(expenditure_total)):
    expenditure_mean.append(((expenditure_total.iloc[i])[3:9]).mean())

In [164]:
expenditure_total["expenditure_mean"] = expenditure_mean

In [165]:
expenditure_total = expenditure_total.drop_duplicates(subset= "country")

In [166]:
expenditure_total

,country,institute_type,direct_expenditure_type,1995,2000,2005,2009,2010,2011,expenditure_std
222,OECD Average,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,6.1,2.490315
223,Australia,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,5.8,2.367840
224,Austria,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,5.7,2.327015
225,Belgium,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,6.6,2.694439
226,Canada,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
227,Chile,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,6.4,2.612789
228,Czech Republic,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,5.0,2.041241
229,Denmark,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,7.9,3.225161
230,Estonia,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,5.5,2.245366
231,Finland,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,6.5,2.653614


In [167]:
expenditure_times = times.merge(expenditure_total)

In [168]:
fig = plotly.express.scatter(data_frame=expenditure_times, x="world_rank", y="expenditure_std", color="expenditure_std", 
                   hover_name="university_name", hover_data=["country","teaching","international","research","citations","income","num_students","institute_type","direct_expenditure_type","total_score"],
                   animation_frame="year",color_discrete_sequence="income")
fig.show()
fig.write_html("Expenditure VS Times world rank.html",auto_play=False)

In [169]:
expenditure_cwur = cwur.merge(expenditure_total)

In [170]:
expenditure_cwur.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,...,year,institute_type,direct_expenditure_type,1995,2000,2005,2009,2010,2011,expenditure_std
0,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,...,2012,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,6.4,2.612789
1,7,University of Oxford,United Kingdom,2,13,28,9,15,13,19,...,2012,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,6.4,2.612789
2,28,Imperial College London,United Kingdom,3,84,73,35,26,26,29,...,2012,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,6.4,2.612789
3,31,University College London,United Kingdom,4,35,101,45,27,23,33,...,2012,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,6.4,2.612789
4,60,University of Edinburgh,United Kingdom,1,71,101,43,64,41,70,...,2012,All Institutions,Total,0.0,0.0,0.0,0.0,0.0,6.4,2.612789


In [171]:
fig = plotly.express.scatter(data_frame=expenditure_cwur, x="world_rank", y="expenditure_std", color="expenditure_std", 
                   hover_name="university_name", hover_data=["country","national_rank","quality_of_education","alumni_employment","citations","influence","quality_of_faculty","institute_type","direct_expenditure_type"],
                   animation_frame="year",color_discrete_sequence="income")
fig.show()
fig.write_html("Expenditure VS Cwur world rank.html",auto_play=False)